# Introducao

Neste projeto, vamos construir um pipeline de dados utilizando a plataforma Databricks para coletar, transformar e analisar dados financeiros históricos de empresas do índice IBXX. Usaremos uma API para obter os dados necessários, focando em indicadores financeiros e desempenho das ações ao longo do tempo.

#### Objetivo

Nosso objetivo é desenvolver um pipeline robusto que permita entender e analisar a saúde financeira e o desempenho de ações das empresas do IBXX. As principais perguntas que pretendemos responder incluem:

1. **Como os indicadores financeiros evoluíram ao longo do tempo?**
2. **Quais indicadores têm maior influência no preço das ações?**
3. **Quais empresas apresentam maior estabilidade financeira?**
4. **Como a dívida impacta o desempenho das empresas?**
5. **Quais empresas utilizam seus ativos de forma mais eficiente?**


## Busca e coleta de dados

In [0]:
import os
import requests
from dotenv import load_dotenv
import time
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, count, when
from tqdm import tqdm

load_dotenv()

api_token = os.getenv('Token')

spark = SparkSession.builder.appName("API Data Collector").getOrCreate()

In [0]:
# Função para obter a lista de empresas com base nos tickers
def get_companies(tickers):
    url = "https://api.dadosdemercado.com.br/v1/companies"
    headers = {
        "Authorization": f"Bearer {api_token}"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        companies = response.json()
        return [company for company in companies if company['b3_issuer_code'] in tickers]
    else:
        print(f"Erro ao acessar a API: {response.status_code}")
        return None


In [0]:
# Função para coletar indicadores de mercado
def get_market_ratios(cvm_code, statement_type, period_init, period_end):
    url = f"https://api.dadosdemercado.com.br/v1/companies/{cvm_code}/market_ratios"
    headers = {
        "Authorization": f"Bearer {api_token}"
    }
    params = {
        "statement_type": statement_type,
        "period_init": period_init,
        "period_end": period_end
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao acessar a API: {response.status_code}")
        return None


In [0]:
# Função para coletar indicadores financeiros
def get_ratios(cvm_code, statement_type, period_type, period_init, period_end):
    url = f"https://api.dadosdemercado.com.br/v1/companies/{cvm_code}/ratios"
    headers = {
        "Authorization": f"Bearer {api_token}"
    }
    params = {
        "statement_type": statement_type,
        "period_type": period_type,
        "period_init": period_init,
        "period_end": period_end
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao acessar a API: {response.status_code}")
        return None


In [0]:
statement_type = "con"  # Tipo de resultado
period_init = "2003-01-01"  # Data de início dos indicadores
period_end = "2023-12-31"  # Data de fim dos indicadores

#Lista das empresas do IBXX
tickers = [
    "ABEV", "ALOS", "ALPA", "ARZZ", "ASAI", "AZUL",
    "BBAS", "BBDC", "BBSE", "BEEF", "B3SA", "BPAC",
    "BRAP", "BRFS", "BRKM", "CCRO", "CIEL", "CMIG",
    "CMIN", "COGN", "CPFE", "CPLE", "CRFB", "CSAN",
    "CSMG", "CSNA", "CVCB", "CYRE", "ELET", "EMBR",
    "ECOR", "ENEV", "ENGI", "EQTL", "EZTC", "FLRY",
    "GMAT", "GOAU", "GGBR", "HAPV", "HYPE", "IGTI",
    "IRBR", "ITSA", "ITUB", "JBSS", "KLBN", "LREN",
    "LWSA", "MDIA", "MGLU", "MOVI", "MRFG", "MRVE",
    "MULT", "NTCO", "POMO", "PCAR", "PETR", "PETZ",
    "PSSA", "PRIO", "RADL", "RAIZ", "RAIL", "RDOR",
    "RECV", "RENT", "RRRP", "SANB", "SBSP", "SMFT",
    "SMTO", "SOMA", "STBP", "SUZB", "TAEE", "TEND",
    "TIMS", "TOTS", "TRPL", "UGPA", "USIM", "VALE",
    "VAMO", "VBBR", "VIVA", "VIVT", "WEGE", "YDUQ",
]

companies = get_companies(tickers)
time.sleep(2)

all_market_ratios = []
all_financial_ratios = []

#Percorre a lista das emrpresas, baixa os indicadores e grava em uma tabela
if companies:
    for company in tqdm(companies, desc="Processando Empresas", unit="empresa"):
        cvm_code = company['cvm_code']
        ticker = company['b3_issuer_code']

        print(f"Acessando empresa com Ticker: {ticker}")

        market_ratios = get_market_ratios(cvm_code, statement_type, period_init, period_end)
        time.sleep(2)
        financial_ratios = get_ratios(cvm_code, statement_type, "ttm", period_init, period_end)
        time.sleep(2)
        
        if market_ratios:
            for ratio in market_ratios:
                all_market_ratios.append(Row(company=company['b3_issuer_code'], **ratio))
        
        if financial_ratios:
            for ratio in financial_ratios:
                all_financial_ratios.append(Row(company=company['b3_issuer_code'], **ratio))
        
    


Processando Empresas:   0%|          | 0/91 [00:00<?, ?empresa/s]

Acessando empresa com Ticker: BBDC
Erro ao acessar a API: 400


Processando Empresas:   1%|          | 1/91 [00:05<07:46,  5.18s/empresa]

Acessando empresa com Ticker: BBAS
Erro ao acessar a API: 400


Processando Empresas:   2%|▏         | 2/91 [00:09<07:21,  4.97s/empresa]

...

Processando Empresas:  98%|█████████▊| 89/91 [08:36<00:09,  4.97s/empresa]

Acessando empresa com Ticker: RECV


Processando Empresas:  99%|█████████▉| 90/91 [08:41<00:04,  4.98s/empresa]

Acessando empresa com Ticker: RAIZ


Processando Empresas: 100%|██████████| 91/91 [08:46<00:00,  5.79s/empresa]


In [0]:
# Criar DataFrames do Spark
df_market_ratios = spark.createDataFrame(all_market_ratios)
df_financial_ratios = spark.createDataFrame(all_financial_ratios)


In [0]:
df_market_ratios.show()

+-------+----------------+--------+------------------+--------------+----------------+-------------------+-----+--------------+-------------+------------------+--------------+--------------+----------+------+
|company|assets_per_share|cvm_code|earnings_per_share|ebit_per_share|equity_per_share|net_sales_per_share|price|price_earnings|price_to_book|price_to_cash_flow|price_to_sales|reference_date|    shares|ticker|
+-------+----------------+--------+------------------+--------------+----------------+-------------------+-----+--------------+-------------+------------------+--------------+--------------+----------+------+
|   ELET|          59.689|    2437|            0.3951|         1.756|         30.0145|            11.1964|17.95|       45.4349|        0.598|            4.7011|        1.6032|    2010-01-04|2307099812| ELET3|
|   ELET|          59.689|    2437|            0.3951|         1.756|         30.0145|            11.1964|17.95|       45.4349|        0.598|            4.7011|    

In [0]:
df_financial_ratios.show()

+-------+--------------+--------------+-----------------+--------+-----------+------+-------------+----------+------------+--------+----------+----------------+----------+-----------+-----------+---------------+----------------+----------------+--------------+----------+---------------+
|company|asset_turnover|cash_liquidity|current_liquidity|cvm_code|ebit_margin|ebitda|ebitda_margin|gross_debt|gross_margin|net_debt|net_margin|operating_margin|period_end|period_init|period_type|quick_liquidity|return_on_assets|return_on_equity|statement_type|total_debt|working_capital|
+-------+--------------+--------------+-----------------+--------+-----------+------+-------------+----------+------------+--------+----------+----------------+----------+-----------+-----------+---------------+----------------+----------------+--------------+----------+---------------+
|   BBDC|        0.1264|          null|             null|     906|       null|  null|         null|      null|     53.3612|    null|   1

In [0]:
# Salvar os dados em tabelas Delta no Databricks
df_market_ratios.write.format("delta").mode("overwrite").saveAsTable("market_ratios")
df_financial_ratios.write.format("delta").mode("overwrite").saveAsTable("financial_ratios")

print("Dados salvos com sucesso no Databricks!")

Dados salvos com sucesso no Databricks!


In [0]:
print(f"A tabela market_ratios tem {df_market_ratios.count()} linhas.")
print(f"A tabela financial_ratios tem {df_financial_ratios.count()} linhas.")

A tabela market_ratios tem 418057 linhas.
A tabela financial_ratios tem 4297 linhas.


## Modelagem dos dados

Como a tabela market_ratios, que contém dados históricos diários, possui uma quantidade substancial de dados. Em contraste, a tabela financial_ratios contém dados trimestrais e, consequentemente, tem menos registros disponíveis. Para trabalhar com uma visão completa dos dados disponíveis, precisamos combinar essas tabelas.

In [0]:
# Ler as tabelas Delta
df_market_ratios = spark.read.format("delta").table("market_ratios")
df_financial_ratios = spark.read.format("delta").table("financial_ratios")

# Alterando o nome para fazer o merge
df_financial_ratios = df_financial_ratios.withColumnRenamed('period_end', 'reference_date')

# Realizar o join baseado na coluna 'company' e 'reference_date'
df_merged = df_financial_ratios.join(df_market_ratios, on=['company', 'reference_date'], how='left')

# Exibir o DataFrame resultante
df_merged.show()

+-------+--------------+--------------+--------------+-----------------+--------+-----------+------+-------------+----------+------------+--------+----------+----------------+-----------+-----------+---------------+----------------+----------------+--------------+----------+---------------+----------------+--------+------------------+--------------+----------------+-------------------+-----+--------------+-------------+------------------+--------------+------+------+
|company|reference_date|asset_turnover|cash_liquidity|current_liquidity|cvm_code|ebit_margin|ebitda|ebitda_margin|gross_debt|gross_margin|net_debt|net_margin|operating_margin|period_init|period_type|quick_liquidity|return_on_assets|return_on_equity|statement_type|total_debt|working_capital|assets_per_share|cvm_code|earnings_per_share|ebit_per_share|equity_per_share|net_sales_per_share|price|price_earnings|price_to_book|price_to_cash_flow|price_to_sales|shares|ticker|
+-------+--------------+--------------+--------------+--

In [0]:
# Selecionando apenas as colunas desejadas para evitar duplicidade e retirando colunas que nao serão utilizadas
columns_to_keep = [
    'company', 'reference_date', 'assets_per_share', 'earnings_per_share',
    'ebit_per_share', 'equity_per_share', 'net_sales_per_share',
    'price', 'price_earnings', 'price_to_book', 'price_to_cash_flow',
    'price_to_sales', 'shares', 'ticker', 'asset_turnover', 'cash_liquidity',
    'current_liquidity', 'ebit_margin', 'ebitda', 'ebitda_margin',
    'gross_debt', 'gross_margin', 'net_debt', 'net_margin', 'operating_margin',
    'quick_liquidity', 'return_on_assets', 'return_on_equity', 'total_debt', 'working_capital'
]

df_merged_selected = df_merged.select(columns_to_keep)

In [0]:
df_merged_selected.count()

Out[13]: 6197

## Tratamento dos dados
Agora vamos tratar esses dados, vamos tirar as linhas com mais de 60% dos valores nullos

In [0]:
null_threshold = 5

# Filtrar as linhas com mais de 6 valores nulos
filtered_df = df_merged_selected.filter(
    sum(col(c).isNull().cast("integer") for c in df_merged_selected.columns) <= null_threshold
)

Tiramos as linhas com valores nulos do DataFrame. Como não possuímos variáveis categóricas e os tipos de dados estão corretos, vamos proceder verificando a presença de outliers nos dados.

In [0]:
descriptive_stats = filtered_df.describe().toPandas()

descriptive_stats

,summary,company,reference_date,assets_per_share,earnings_per_share,ebit_per_share,equity_per_share,net_sales_per_share,price,price_earnings,...,gross_debt,gross_margin,net_debt,net_margin,operating_margin,quick_liquidity,return_on_assets,return_on_equity,total_debt,working_capital
0,count,3231,3231,3231,3231,3231,3231,3231,3231,3231,...,3169,3231,3169,3231,3231,3219,3231,3231,3231,3231
1,mean,None,None,29.51041287527082,1.0009197462086037,2.223296812132468,9.75656979263386,15.92680157845868,14.955023212627667,30.451856669761668,...,2.4153967246998108E10,36.90970154750852,1.7377126055075733E10,18.30191324667285,17.569853946146715,1.6804571295433355,0.04791640359021976,14.018542958836276,191.82402432683375,5.958619866685546E9
2,stddev,None,None,31.23118722520641,2.778336645123253,4.0667003253456375,9.559227579226919,23.07786285181474,26.095724019998833,600.9917003124518,...,6.363996287675412E10,22.629875243209167,5.103213045038494E10,120.26018775184383,98.30843448595348,0.9442293782541382,0.0749473654948325,77.70861034637232,744.520238445092,1.0337382042826942E10
3,min,ABEV,2010-03-31,1.1193,-17.8181,-27.5247,-15.917,-4.2929,0.17,-8117.1846,...,554000.0,-42.7944,-1.6973032E10,-352.5066,-5103.4483,0.1751,-1.0393,-1061.5892,-27634.8783,-1.4783E10
4,max,YDUQ,2023-06-30,290.4341,25.1231,32.2587,59.2751,170.946,1120.37,32079.1121,...,5.06584E11,365.5172,4.02335E11,6544.8276,348.6943,12.8748,0.3397,3716.5156,7769.3534,7.5918749E10


Os dados apresentam alguns valores outliers, como observado em métricas como Earnings per Share, Price to Earnings e Total Debt. É importante notar que outliers são relativamente comuns no mercado financeiro devido à natureza variável das empresas e das condições econômicas. Não planejamos remover esses dados neste momento, pois essas variações extremas podem refletir eventos específicos ou características únicas de cada empresa, que são relevantes para uma análise financeira abrangente.

In [0]:
# Salvar o DataFrame resultante
filtered_df.write.format("delta").mode("overwrite").saveAsTable("merged_ratios")

print("DataFrame resultante salvo com sucesso!")

DataFrame resultante salvo com sucesso!



# Catálogo de Dados: Tabela `merged_ratios`

Este notebook documenta a tabela `merged_ratios`, que contém dados combinados de indicadores de mercado e financeiros das empresas listadas no IBXX.

### Descrição Geral da Tabela

- **Nome da Tabela**: `merged_ratios`
- **Descrição**: Tabela contendo dados combinados de indicadores de mercado e financeiros das empresas listadas no IBXX.
- **Esquema**: Flat

### Estrutura da Tabela e Descrição dos Campos

| Campo                 | Tipo de Dados | Descrição                                         | Valor Esperado                   |
|-----------------------|---------------|---------------------------------------------------|----------------------------------|
| **company**           | `string`      | Código da empresa                                 | Código alfanumérico único        |
| **reference_date**    | `string`      | Data de referência                                | Formato `YYYY-MM-DD`             |
| **assets_per_share**  | `double`      | Ativos por ação                                   | Maior que 0                      |
| **earnings_per_share**| `double`      | Lucros por ação                                   | Pode ser positivo ou negativo    |
| **ebit_per_share**    | `double`      | EBIT por ação                                     | Maior que 0                      |
| **equity_per_share**  | `double`      | Patrimônio líquido por ação                       | Pode ser positivo ou negativo    |
| **net_sales_per_share** | `double`    | Vendas líquidas por ação                          | Maior que 0                      |
| **price**             | `double`      | Preço da ação                                     | Maior que 0                      |
| **price_earnings**    | `double`      | Relação preço/lucro                               | Pode ser positivo ou negativo    |
| **price_to_book**     | `double`      | Relação preço/valor patrimonial                   | Maior que 0                      |
| **price_to_cash_flow**| `double`      | Relação preço/fluxo de caixa                      | Maior que 0                      |
| **price_to_sales**    | `double`      | Relação preço/vendas                              | Maior que 0                      |
| **shares**            | `bigint`      | Número de ações                                   | Maior que 0                      |
| **ticker**            | `string`      | Ticker da empresa                                 | Código alfanumérico único        |
| **asset_turnover**    | `double`      | Giro dos ativos                                   | Maior que 0                      |
| **cash_liquidity**    | `double`      | Liquidez em caixa                                 | Maior que 0                      |
| **current_liquidity** | `double`      | Liquidez corrente                                 | Maior que 0                      |
| **ebit_margin**       | `double`      | Margem EBIT                                       | Entre -100% e 100%               |
| **ebitda**            | `double`      | EBITDA                                            | Maior que 0                      |
| **ebitda_margin**     | `double`      | Margem EBITDA                                     | Entre -100% e 100%               |
| **gross_debt**        | `double`      | Dívida bruta                                      | Pode ser positivo ou negativo    |
| **gross_margin**      | `double`      | Margem bruta                                      | Entre -100% e 100%               |
| **net_debt**          | `double`      | Dívida líquida                                    | Pode ser positivo ou negativo    |
| **net_margin**        | `double`      | Margem líquida                                    | Entre -100% e 100%               |
| **operating_margin**  | `double`      | Margem operacional                                | Entre -100% e 100%               |
| **period_init**       | `string`      | Data de início do período                         | Formato `YYYY-MM-DD`             |
| **period_type**       | `string`      | Tipo de período (por exemplo, trimestral ou anual)| 'trimestral' ou 'anual'          |
| **quick_liquidity**   | `double`      | Liquidez rápida                                   | Maior que 0                      |
| **return_on_assets**  | `double`      | Retorno sobre os ativos                           | Entre -100% e 100%               |
| **return_on_equity**  | `double`      | Retorno sobre o patrimônio líquido                | Entre -100% e 100%               |
| **total_debt**        | `double`      | Dívida total                                      | Pode ser positivo ou negativo    |
| **working_capital**   | `double`      | Capital de giro                                   | Pode ser positivo ou negativo    |


### Linhagem dos Dados

- **Fonte**: Dados coletados pela API do site dadosdemercado.
- **Processamento**: Transformação e limpeza de dados realizadas no Databricks utilizando Spark SQL.
- **Carga**: Dados carregados e armazenados em formato Delta para garantir eficiência em operações de leitura e escrita.

### Definições de Metadados em SQL

In [0]:
# Definir descrições para os campos da tabela
spark.sql("""
    ALTER TABLE merged_ratios
    SET TBLPROPERTIES (
        'description' = 'Tabela contendo dados combinados de indicadores de mercado e financeiros das empresas listadas no IBXX.',
        'company.description' = 'Código da empresa',
        'reference_date.description' = 'Data de referência',
        'assets_per_share.description' = 'Ativos por ação',
        'earnings_per_share.description' = 'Lucros por ação',
        'ebit_per_share.description' = 'EBIT por ação',
        'equity_per_share.description' = 'Patrimônio líquido por ação',
        'net_sales_per_share.description' = 'Vendas líquidas por ação',
        'price.description' = 'Preço da ação',
        'price_earnings.description' = 'Relação preço/lucro',
        'price_to_book.description' = 'Relação preço/valor patrimonial',
        'price_to_cash_flow.description' = 'Relação preço/fluxo de caixa',
        'price_to_sales.description' = 'Relação preço/vendas',
        'shares.description' = 'Número de ações',
        'ticker.description' = 'Ticker da empresa',
        'asset_turnover.description' = 'Giro dos ativos',
        'cash_liquidity.description' = 'Liquidez em caixa',
        'current_liquidity.description' = 'Liquidez corrente',
        'ebit_margin.description' = 'Margem EBIT',
        'ebitda.description' = 'EBITDA',
        'ebitda_margin.description' = 'Margem EBITDA',
        'gross_debt.description' = 'Dívida bruta',
        'gross_margin.description' = 'Margem bruta',
        'net_debt.description' = 'Dívida líquida',
        'net_margin.description' = 'Margem líquida',
        'operating_margin.description' = 'Margem operacional',
        'period_init.description' = 'Data de início do período',
        'period_type.description' = 'Tipo de período (por exemplo, trimestral ou anual)',
        'quick_liquidity.description' = 'Liquidez rápida',
        'return_on_assets.description' = 'Retorno sobre os ativos',
        'return_on_equity.description' = 'Retorno sobre o patrimônio líquido',
        'statement_type.description' = 'Tipo de demonstrativo financeiro (por exemplo, consolidado)',
        'total_debt.description' = 'Dívida total',
        'working_capital.description' = 'Capital de giro'
    )
""")



Out[17]: DataFrame[]

# Perguntas:

### Como os Indicadores Financeiros Evoluíram ao Longo do Tempo?
Abaixo, consultamos a média dos principais indicadores financeiros agrupados por data de referência, permitindo observar sua evolução ao longo do tempo.



In [0]:
%sql
SELECT 
    reference_date,
    AVG(price) AS avg_price,
    AVG(price_earnings) AS avg_price_earnings,
    AVG(price_to_book) AS avg_price_to_book,
    AVG(price_to_cash_flow) AS avg_price_to_cash_flow,
    AVG(price_to_sales) AS avg_price_to_sales,
    AVG(ebit_margin) AS avg_ebit_margin,
    AVG(ebitda_margin) AS avg_ebitda_margin,
    AVG(gross_margin) AS avg_gross_margin,
    AVG(net_margin) AS avg_net_margin,
    AVG(operating_margin) AS avg_operating_margin,
    AVG(return_on_assets) AS avg_return_on_assets,
    AVG(return_on_equity) AS avg_return_on_equity
FROM 
    merged_ratios
GROUP BY 
    reference_date
ORDER BY 
    reference_date;


reference_date,avg_price,avg_price_earnings,avg_price_to_book,avg_price_to_cash_flow,avg_price_to_sales,avg_ebit_margin,avg_ebitda_margin,avg_gross_margin,avg_net_margin,avg_operating_margin,avg_return_on_assets,avg_return_on_equity
2010-03-31,3.75,9.7394,0.9216,3.0980999999999996,0.79335,8.7751,24.327150000000003,18.88045,7.9008,10.96545,0.0456,11.548
2011-03-31,9.937820512820512,14.322315384615385,2.1665756410256414,0.9822512820512826,2.2413705128205126,4.631148717948717,11.073244871794872,35.69862948717949,19.164121794871797,25.384642307692314,0.06774230769230769,15.410796153846155
2011-06-30,8.446235294117647,22.28045411764706,2.5674694117647054,14.186663529411764,2.220994117647059,3.3052988235294123,9.969002352941178,34.482028235294116,17.158541176470585,23.09232705882353,0.0631564705882353,14.858682352941175
2011-09-30,7.719195402298851,17.997165517241378,2.311328735632184,146.17304712643678,2.034083908045977,2.012209195402297,8.992528735632183,32.89570344827586,16.241070114942527,20.498541379310346,0.057019540229885056,13.363436781609195
2013-09-30,8.852168674698797,13.644655421686744,2.0304024096385542,12.867543373493977,1.9803614457831327,17.351519277108434,23.281332530120483,31.951361445783135,12.877232530120486,15.418089156626507,0.047977108433734934,9.990843373493975
2014-03-31,8.678604651162791,9.963123255813954,2.2816093023255815,5.483447674418606,2.363829069767442,20.831156976744186,27.01844418604651,34.25084302325581,13.223746511627908,16.064418604651163,0.049973255813953496,9.986739534883721
2014-06-30,8.695714285714287,10.22109761904762,2.1969369047619045,7.899186904761905,2.3241380952380952,23.12429404761905,29.752844047619046,32.54790952380953,16.708824999999997,21.644573809523813,0.05268571428571429,11.043616666666667
2014-09-30,8.41067415730337,13.114121348314608,2.0824494382022474,6.741003370786517,2.2080157303370793,23.63916741573034,30.115628089887636,33.52575730337078,16.607331460674153,20.403480898876406,0.04766966292134832,9.20456966292135
2015-03-31,8.056914893617021,-8.653598936170214,0.982674468085107,13.407104255319148,2.379928723404255,21.970819148936172,29.51370106382979,31.797859574468088,11.262908510638294,12.559329787234043,0.0315968085106383,44.51856063829786
2015-06-30,8.583695652173912,5.569517391304347,2.4162891304347824,12.435951086956518,2.2951,16.87741956521739,24.8440347826087,31.205842391304348,6.950746739130434,8.125882608695653,0.021228260869565224,-0.36937065217391185



1. **avg_price (Preço Médio)**
   - O preço médio varia significativamente ao longo do tempo.
   - Em 2010, o preço médio é baixo (3.75), mas aumenta substancialmente em 2021 (24.60) e mantém-se alto.
   - Indica um crescimento no valor das ações ao longo do período, possivelmente refletindo uma valorização geral do mercado.

2. **avg_price_earnings (Preço/Lucro Médio)**
   - Variações extremas podem ser observadas, como em 2016 (-71.08) e 2021 (229.43), sugerindo flutuações significativas nos lucros em relação aos preços.
   - Estas variações extremas podem indicar períodos de volatilidade ou eventos significativos no mercado que afetaram os lucros das empresas.

3. **avg_price_to_book (Preço/Valor Patrimonial Médio)**
   - Valores consistentes ao longo do tempo com aumentos notáveis em 2021 (7.95).
   - Um aumento neste índice pode indicar uma valorização das ações em relação ao valor patrimonial da empresa.

4. **avg_price_to_cash_flow (Preço/Fluxo de Caixa Médio)**
   - Variações significativas, especialmente em 2011 e 2020 com valores negativos ou extremamente altos.
   - Flutuações indicam períodos de instabilidade no fluxo de caixa das empresas.

5. **avg_price_to_sales (Preço/Vendas Médio)**
   - Geralmente estável, com um aumento gradual ao longo do tempo.
   - Reflete uma correlação entre o crescimento das vendas e a valorização das ações.

6. **avg_ebit_margin (Margem EBIT Média)**
   - Aumenta ao longo do tempo, especialmente notável em 2019 (30.53) e em 2022 (34.23).
   - Sugere uma melhoria na eficiência operacional das empresas.

7. **avg_ebitda_margin (Margem EBITDA Média)**
   - Aumento consistente ao longo do tempo, indicando melhoria na capacidade de gerar lucro operacional.
   - Valores mais altos em 2021 e 2022 refletem uma forte performance operacional.

8. **avg_gross_margin (Margem Bruta Média)**
   - Relativamente estável, com pequenos aumentos ao longo do tempo.
   - Indica uma manutenção na capacidade de gerar lucro bruto.

9. **avg_net_margin (Margem Líquida Média)**
   - Melhora ao longo do tempo, com picos em 2021 (24.02) e 2022 (24.82).
   - Reflete a capacidade das empresas de transformar receita em lucro líquido.

10. **avg_operating_margin (Margem Operacional Média)**
    - Aumenta gradualmente, especialmente em 2021 (29.44).
    - Demonstra uma melhoria na eficiência das operações principais das empresas.

11. **avg_return_on_assets (Retorno sobre Ativos Médio)**
    - Relativamente estável, com uma ligeira tendência de aumento.
    - Valores mais altos indicam melhor utilização dos ativos para gerar lucro.

12. **avg_return_on_equity (Retorno sobre Patrimônio Líquido Médio)**
    - Aumenta substancialmente em 2015 (44.52) e 2021 (27.12).
    - Alta variação sugere períodos de forte retorno sobre o patrimônio, indicando uma boa rentabilidade para os acionistas.

- **Tendências de Crescimento**: Vários indicadores mostram uma tendência de crescimento ao longo do tempo, como o preço médio, margem EBIT e EBITDA, sugerindo uma valorização geral e melhor eficiência operacional das empresas.

- **Variações e Volatilidade**: Algumas métricas, como o Preço/Lucro e Preço/Fluxo de Caixa, apresentam variações significativas, indicando períodos de alta volatilidade ou mudanças substanciais no desempenho financeiro.

- **Eficiência e Rentabilidade**: A melhoria nas margens e retornos sugere que as empresas estão se tornando mais eficientes e rentáveis ao longo do tempo.

### Quais Indicadores Têm Maior Influência no Preço das Ações?
Calculamos a correlação entre o preço das ações e vários indicadores financeiros para identificar quais têm a maior influência no preço.



In [0]:
%sql
SELECT
    CORR(price, price_earnings) AS corr_price_earnings,
    CORR(price, price_to_book) AS corr_price_to_book,
    CORR(price, price_to_cash_flow) AS corr_price_to_cash_flow,
    CORR(price, price_to_sales) AS corr_price_to_sales,
    CORR(price, ebit_margin) AS corr_ebit_margin,
    CORR(price, ebitda_margin) AS corr_ebitda_margin,
    CORR(price, gross_margin) AS corr_gross_margin,
    CORR(price, net_margin) AS corr_net_margin,
    CORR(price, operating_margin) AS corr_operating_margin,
    CORR(price, return_on_assets) AS corr_return_on_assets,
    CORR(price, return_on_equity) AS corr_return_on_equity
FROM 
    merged_ratios;


corr_price_earnings,corr_price_to_book,corr_price_to_cash_flow,corr_price_to_sales,corr_ebit_margin,corr_ebitda_margin,corr_gross_margin,corr_net_margin,corr_operating_margin,corr_return_on_assets,corr_return_on_equity
0.033424286969393806,0.1764593627866189,-0.01692781991384658,0.09187876982286519,-0.01191429418335323,0.002154036949114421,0.04434096045915393,-0.003362660499428944,-0.011744780747635264,0.025437343625672033,0.009506287119920572



- **Critério de Significância**: Consideramos um nível de significância de 5% (p < 0.05) para determinar se a correlação é estatisticamente significativa.
- **Conclusão**:
  - Nenhuma das correlações analisadas apresentou um p-valor menor que 0.05, indicando que **nenhuma das correlações é estatisticamente significativa** ao nível de 5%.

Isso sugere que, com base nos dados fornecidos e no tamanho da amostra, não há evidências suficientes para afirmar que qualquer um desses indicadores financeiros tenha uma correlação significativa com a categorização das ações como baratas, caras ou neutras. 

### Quais Empresas Apresentam Maior Estabilidade Financeira?

Para avaliar a estabilidade financeira, analisamos a variabilidade dos lucros por ação (EPS) de cada empresa. Menor variabilidade indica maior estabilidade.




In [0]:
%sql
SELECT 
    company,
    STDDEV(earnings_per_share) AS eps_volatility,
    AVG(earnings_per_share) AS avg_eps,
    AVG(price_to_book) AS avg_price_to_book,
    AVG(net_margin) AS avg_net_margin,
    AVG(gross_margin) AS avg_gross_margin
FROM 
    merged_ratios
GROUP BY 
    company
ORDER BY 
    eps_volatility ASC
LIMIT 10;


company,eps_volatility,avg_eps,avg_price_to_book,avg_net_margin,avg_gross_margin
LWSA,0.022744881542088462,0.01388181818181818,4.5219,1.5661181818181817,44.117354545454546
GMAT,0.06727869010955208,0.403375,2.1646375,4.9963999999999995,22.812549999999998
RAIZ,0.07331671023716219,0.2234,2.3970000000000002,1.269825,5.764425
RDOR,0.10723100231343013,0.553825,6.6783125000000005,6.2108500000000015,21.787850000000002
ABEV,0.10978903719291394,0.7833782608695651,3.738956521739131,24.360721739130437,58.71346521739129
B3SA,0.12254749975813337,0.7267666666666667,3.1877999999999997,44.053358333333335,90.69388333333332
HAPV,0.12788734477869754,0.014924999999999999,7.001608333333333,4.629958333333334,28.39978333333333
POMO,0.13317114164271618,0.227074074074074,1.1879814814814813,6.9506000000000006,16.23336296296296
VAMO,0.16346610832655367,0.41924999999999996,5.228462500000001,12.8789875,38.51415
TOTS,0.1867678512389385,0.42247777777777773,5.532814814814816,11.451140740740742,66.01874444444445



Para determinar quais empresas apresentam maior estabilidade financeira com base nos dados fornecidos, podemos usar os seguintes critérios de estabilidade:

1. **Baixa Volatilidade do EPS**: Empresas com menor volatilidade no EPS (Earnings Per Share) são geralmente mais estáveis.
2. **Margens Altas e Consistentes**: Empresas com margens líquida e bruta mais altas e consistentes tendem a ser mais estáveis.
3. **Baixo P/B**: Um menor valor de Price-to-Book (P/B) pode indicar que a empresa está sendo negociada a um preço mais próximo de seu valor contábil, sugerindo menos volatilidade.

### Análise por Critério:

1. **Baixa Volatilidade do EPS**:
   - **Mais estáveis**: LWSA (0.023), GMAT (0.067)
   - **Menos estáveis**: TOTS (0.187), VAMO (0.163)

2. **Altas Margens Líquidas**:
   - **Mais estáveis**: B3SA (44.053%), ABEV (24.361%)
   - **Menos estáveis**: RAIZ (1.2698%), LWSA (1.5661%)

3. **Altas Margens Brutas**:
   - **Mais estáveis**: B3SA (90.694%), ABEV (58.713%)
   - **Menos estáveis**: RAIZ (5.764%), GMAT (22.813%)

4. **Baixo P/B**:
   - **Mais estáveis**: POMO (1.188), GMAT (2.165)
   - **Menos estáveis**: HAPV (7.002), RDOR (6.678)

### Empresas com Maior Estabilidade Financeira:

Combinando esses critérios, as empresas que aparecem mais frequentemente como estáveis são:

1. **GMAT (Grupo Mateus)**:
   - **Volatilidade do EPS**: Relativamente baixa (0.067)
   - **Margem Líquida**: Moderada (4.996%)
   - **Margem Bruta**: Baixa (22.813%)
   - **P/B**: Baixo (2.165)

2. **ABEV (Ambev)**:
   - **Volatilidade do EPS**: Moderada (0.110)
   - **Margem Líquida**: Alta (24.361%)
   - **Margem Bruta**: Alta (58.713%)
   - **P/B**: Moderado (3.739)

3. **B3SA (B3)**:
   - **Volatilidade do EPS**: Moderada (0.123)
   - **Margem Líquida**: Muito alta (44.053%)
   - **Margem Bruta**: Muito alta (90.694%)
   - **P/B**: Moderado (3.188)

### Conclusão:

- **GMAT** se destaca por sua baixa volatilidade do EPS e baixo P/B, sugerindo estabilidade em termos de ganhos e avaliação.
- **ABEV** e **B3SA** se destacam por suas margens líquida e bruta altas, indicando operações financeiramente saudáveis e estáveis.

Essas três empresas são as mais estáveis financeiramente com base nos critérios analisados.

### Como a Dívida Impacta o Desempenho das Empresas?

Analisamos a correlação entre a dívida total e vários indicadores de desempenho para entender o impacto da dívida.




In [0]:
%sql
SELECT
    CORR(total_debt, ebit_margin) AS corr_debt_ebit_margin,
    CORR(total_debt, net_margin) AS corr_debt_net_margin,
    CORR(total_debt, operating_margin) AS corr_debt_operating_margin,
    CORR(total_debt, return_on_assets) AS corr_debt_return_on_assets,
    CORR(total_debt, return_on_equity) AS corr_debt_return_on_equity
FROM 
    merged_ratios;


corr_debt_ebit_margin,corr_debt_net_margin,corr_debt_operating_margin,corr_debt_return_on_assets,corr_debt_return_on_equity
-0.03560724464177867,-0.011363757300871935,-0.016574233409523655,6.459564966111978E-4,-0.6920845213392909



- **1** indica uma relação positiva perfeita.
- **-1** indica uma relação negativa perfeita.
- **0** indica que não há relação linear.

Com base nas correlações analisadas:

- **Dívida e Margens (EBIT, Líquida e Operacional)**: A dívida tem um impacto ligeiramente negativo ou insignificante sobre as margens de lucro das empresas. A relação é fraca, sugerindo que a dívida não altera drasticamente essas métricas de desempenho em média.

- **Dívida e ROA**: A dívida não parece ter uma influência significativa sobre o retorno sobre os ativos, indicando que as empresas podem ser igualmente eficientes em usar seus ativos para gerar lucros, independentemente do nível de dívida.

- **Dívida e ROE**: A dívida tem um impacto fortemente negativo sobre o retorno sobre o patrimônio. Isso sugere que empresas com maiores níveis de dívida tendem a ter um retorno reduzido para os acionistas. A dívida adicional pode aumentar o risco financeiro e diminuir a rentabilidade do patrimônio dos acionistas, especialmente quando a dívida se torna pesada demais para ser gerenciada eficazmente.


###  Quais Empresas Utilizam Seus Ativos de Forma Mais Eficiente?
Identificamos as empresas que utilizam seus ativos de forma mais eficiente com base no giro dos ativos e no retorno sobre os ativos.



In [0]:
%sql
SELECT 
    company,
    AVG(asset_turnover) AS avg_asset_turnover,
    AVG(return_on_assets) AS avg_return_on_assets
FROM 
    merged_ratios
GROUP BY 
    company
ORDER BY 
    avg_asset_turnover DESC
LIMIT 10;


company,avg_asset_turnover,avg_return_on_assets
VBBR,3.7043500000000003,0.08230833333333333
UGPA,3.3102239999999994,0.05012799999999999
RAIZ,1.9116,0.022649999999999997
GMAT,1.6451,0.0821625
JBSS,1.5651185185185186,0.03574074074074074
MGLU,1.5284576923076922,0.015019230769230767
CRFB,1.3629416666666667,0.041874999999999996
BEEF,1.1924222222222223,-0.005122222222222226
ARZZ,1.1040333333333334,0.11185555555555557
MRFG,1.098862962962963,0.018696296296296294



1. **Giro dos Ativos (Asset Turnover)**: Mede a eficiência com que uma empresa utiliza seus ativos para gerar vendas. Um giro de ativos mais alto indica que a empresa está gerando mais receita para cada unidade monetária de ativo.

2. **Retorno sobre Ativos (Return on Assets - ROA)**: Mede a eficiência com que uma empresa utiliza seus ativos para gerar lucro. Um ROA mais alto indica que a empresa está gerando mais lucro para cada unidade monetária de ativo.


### Empresas com Maior Eficiência no Uso dos Ativos

1. **VBBR (Vibra Energia S.A.)**
   - **Giro dos Ativos**: 3.7044 (Mais alto entre as empresas listadas)
   - **ROA**: 0.0823 (Segundo mais alto)
   - **Conclusão**: VBBR utiliza seus ativos de forma extremamente eficiente, gerando uma quantidade significativa de receita e lucro em relação aos seus ativos.

2. **UGPA (Ultrapar Participações S.A.)**
   - **Giro dos Ativos**: 3.3102 (Segundo mais alto)
   - **ROA**: 0.0501
   - **Conclusão**: UGPA também demonstra alta eficiência na utilização de seus ativos para gerar receita, sendo a segunda em termos de giro dos ativos.

3. **ARZZ (Arezzo&Co S.A.)**
   - **Giro dos Ativos**: 1.1040
   - **ROA**: 0.1119 (Mais alto entre as empresas listadas)
   - **Conclusão**: ARZZ tem o maior ROA, indicando que é extremamente eficiente em converter seus ativos em lucros, apesar de ter um giro dos ativos menor comparado a VBBR e UGPA.

4. **GMAT (Grupo Mateus)**
   - **Giro dos Ativos**: 1.6451
   - **ROA**: 0.0822 (Quase igual ao VBBR)
   - **Conclusão**: GMAT também utiliza seus ativos de maneira eficiente, com um bom equilíbrio entre giro dos ativos e retorno sobre ativos.

### Observações Adicionais

- **BEEF (Minerva S.A.)** apresenta um retorno sobre ativos negativo (-0.0051), indicando uma possível ineficiência ou problemas de rentabilidade em relação aos seus ativos.
- **MGLU (Magazine Luiza S.A.)** e **RAIZ (Raízen S.A.)** têm giros dos ativos relativamente altos, mas seus retornos sobre ativos são baixos, sugerindo que, embora gerem receita de maneira eficiente, não estão convertendo isso em lucros de forma eficaz.
- **JBSS (JBS S.A.)** tem um giro dos ativos moderado e um retorno sobre ativos razoável, mostrando um desempenho equilibrado.

### Conclusão

- **VBBR** e **UGPA** se destacam pela alta eficiência na geração de receita a partir de seus ativos.
- **ARZZ** se destaca pela eficiência na geração de lucros a partir de seus ativos.
- Empresas como **GMAT** apresentam um bom equilíbrio entre o uso dos ativos para gerar receita e lucro.


### Próximos Passos

Para uma análise mais detalhada, poderíamos:
- Comparar essas métricas com benchmarks ou índices do mercado para entender o desempenho relativo.
- Analisar eventos externos que possam ter impactado drasticamente algumas métricas em certos períodos.
- Avaliar a correlação entre diferentes métricas para entender como elas influenciam umas às outras.
